## Jupyter notebook for USASpending.

##### In this notebook I have loaded multiple csv files which is 33 GB in volume from USASpending. 

In [1]:
import os, glob, pathlib
import math
import shutil
from datetime import datetime
from collections import Counter
import zipfile
import pandas as pd
import dask
import dask.dataframe as dd
from dask.distributed import Client, progress
import psutil
import requests

# speed up suggestion from here - https://stackoverflow.com/questions/51590696/speeding-up-read-csv-in-python-pandas
from multiprocessing import Pool

### Setting up Dask cluster

In [2]:

from distributed import Client, LocalCluster

def Close_Dask_Client_Robust():
    global client
    try:
        client
    except NameError:
        client = None

    if client:
        client.close() # you can use this command to close the Dask processes and then repeat the command above to launch it again

global client

Close_Dask_Client_Robust()

dask_workers = int(psutil.cpu_count(logical=False))
client = Client(n_workers=dask_workers) 

client 

Connection method: Cluster object,Cluster type: LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Status: running,Using processes: True
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 15.59 GiB
Comm: tcp://127.0.0.1:52084,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 15.59 GiB
Comm: tcp://127.0.0.1:52112,Total threads: 2
Dashboard: http://127.0.0.1:52113/status,Memory: 3.90 GiB
Nanny: tcp://127.0.0.1:52088,


In [3]:
# this assumes the jupyter notebook is running in the same higher level directory
# where the USAspending.gov zip archives and the decompressed Zip files are stored

original_cwd = os.getcwd()
download_file_path = os.path.join(original_cwd, "USAspending_Archive_Downloads") #
download_file_path_CSV = os.path.join(original_cwd, "USAspending_Archive_Downloads", "Expanded_CSV_Files")
print(original_cwd)
print(download_file_path)
print(download_file_path_CSV)
#your_path_dummy = 'Users/YourName/YourTopFolder_Documents/YourUSAspendingFolderName'
#print(original_cwd)
#print(os.path.join(your_path_dummy, os.path.basename(download_file_path)))
#print(os.path.join(os.path.join(your_path_dummy, os.path.basename(download_file_path)), os.path.basename(download_file_path_CSV)))

c:\Users\msaeed\Downloads\USAspending_Medium_Blog_Analytics_Series-master
c:\Users\msaeed\Downloads\USAspending_Medium_Blog_Analytics_Series-master\USAspending_Archive_Downloads
c:\Users\msaeed\Downloads\USAspending_Medium_Blog_Analytics_Series-master\USAspending_Archive_Downloads\Expanded_CSV_Files


In [4]:
def Get_Current_Time():
    return datetime.now().strftime("%d/%m/%Y %H:%M:%S")

Get_Current_Time()

'19/09/2021 22:38:00'

In [5]:
# def Enhance_Spending_File(df): # this function addresses known shortcomings in the USAspending archive data
#     print()
#     print(f"{Get_Current_Time()} -> replacing NaN with UNSPECIFIED...")
#     df = df.fillna("UNSPECIFIED") # IMPORTANT - NA fields can affect groupby sums and other problems
#     return df

### You can limit the selection to a subset of the GFY to speed processing

In [5]:
# You can limit the selection to a subset of the GFY to speed processing

ALL_GFY = False # switch to True to override the subset

user_select_GFY = ['FY2021']

if ALL_GFY:
    user_select_GFY = all_GFY_list
    
# print(f"GFY to read for analysis: {sorted(user_select_GFY)}")
# print()

# Create csv_files_list_restricted to streamline downstream data processing

csv_files_list = sorted(glob.glob(os.path.join(download_file_path_CSV, "*.csv")))
print(f'Total number of files: {len(csv_files_list)}')
csv_files_list_restricted = [filename for filename in csv_files_list for GFY in user_select_GFY if GFY in filename]
print(f'Number of selected files: {len(csv_files_list_restricted)}')

Total number of files: 30
Number of selected files: 5


In [6]:
## Key code for read CSV files into pandas and Dask
def Build_DTypes_Dict(filename_list):
    df = pd.read_csv(filename_list[0], nrows=1)
    
    dtype = dict(zip(sorted(df.columns), ['object'] * len(df.columns)))
    
    # ! important - may need to change other fields to category, int, date etc for better memory management
    dtype['federal_action_obligation'] = 'float64'
    #dtype['action_date_fiscal_year'] = 'int'
    
    return dtype

In [7]:
%%time
dtype = Build_DTypes_Dict(csv_files_list_restricted)
ddf = dd.read_csv(csv_files_list_restricted, dtype = dtype)
ddf = ddf.fillna("UNSPECIFIED")

Wall time: 192 ms


In [8]:
%%time
dfx = pd.read_csv(csv_files_list[0], nrows=5)
pd.set_option('display.max_columns', None)
dfx

Wall time: 60.2 ms


,contract_transaction_unique_key,contract_award_unique_key,award_id_piid,modification_number,transaction_number,parent_award_agency_id,parent_award_agency_name,parent_award_id_piid,parent_award_modification_number,federal_action_obligation,total_dollars_obligated,base_and_exercised_options_value,current_total_value_of_award,base_and_all_options_value,potential_total_value_of_award,disaster_emergency_fund_codes_for_overall_award,outlayed_amount_funded_by_COVID-19_supplementals_for_overall_aw,obligated_amount_funded_by_COVID-19_supplementals_for_overall_a,action_date,action_date_fiscal_year,period_of_performance_start_date,period_of_performance_current_end_date,period_of_performance_potential_end_date,ordering_period_end_date,solicitation_date,awarding_agency_code,awarding_agency_name,awarding_sub_agency_code,awarding_sub_agency_name,awarding_office_code,awarding_office_name,funding_agency_code,funding_agency_name,funding_sub_agency_code,funding_sub_agency_name,funding_office_code,funding_office_name,treasury_accounts_funding_this_award,federal_accounts_funding_this_award,object_classes_funding_this_award,program_activities_funding_this_award,foreign_funding,foreign_funding_description,sam_exception,sam_exception_description,recipient_duns,recipient_name,recipient_doing_business_as_name,cage_code,recipient_parent_duns,recipient_parent_name,recipient_country_code,recipient_country_name,recipient_address_line_1,recipient_address_line_2,recipient_city_name,recipient_county_name,recipient_state_code,recipient_state_name,recipient_zip_4_code,recipient_congressional_district,recipient_phone_number,recipient_fax_number,primary_place_of_performance_country_code,primary_place_of_performance_country_name,primary_place_of_performance_city_name,primary_place_of_performance_county_name,primary_place_of_performance_state_code,primary_place_of_performance_state_name,primary_place_of_performance_zip_4,primary_place_of_performance_congressional_district,award_or_idv_flag,award_type_code,award_type,idv_type_code,idv_type,multiple_or_single_award_idv_code,multiple_or_single_award_idv,type_of_idc_code,type_of_idc,type_of_contract_pricing_code,type_of_contract_pricing,award_description,action_type_code,action_type,solicitation_identifier,number_of_actions,inherently_governmental_functions,inherently_governmental_functions_description,product_or_service_code,product_or_service_code_description,contract_bundling_code,contract_bundling,dod_claimant_program_code,dod_claimant_program_description,naics_code,naics_description,recovered_materials_sustainability_code,recovered_materials_sustainability,domestic_or_foreign_entity_code,domestic_or_foreign_entity,dod_acquisition_program_code,dod_acquisition_program_description,information_technology_commercial_item_category_code,information_technology_commercial_item_category,epa_designated_product_code,epa_designated_product,country_of_product_or_service_origin_code,country_of_product_or_service_origin,place_of_manufacture_code,place_of_manufacture,subcontracting_plan_code,subcontracting_plan,extent_competed_code,extent_competed,solicitation_procedures_code,solicitation_procedures,type_of_set_aside_code,type_of_set_aside,evaluated_preference_code,evaluated_preference,research_code,research,fair_opportunity_limited_sources_code,fair_opportunity_limited_sources,other_than_full_and_open_competition_code,other_than_full_and_open_competition,number_of_offers_received,commercial_item_acquisition_procedures_code,commercial_item_acquisition_procedures,small_business_competitiveness_demonstration_program,simplified_procedures_for_certain_commercial_items_code,simplified_procedures_for_certain_commercial_items,a76_fair_act_action_code,a76_fair_act_action,fed_biz_opps_code,fed_biz_opps,local_area_set_aside_code,local_area_set_aside,price_evaluation_adjustment_preference_percent_difference,clinger_cohen_act_planning_code,clinger_cohen_act_planning,materials_supplies_articles_equipment_code,materials_supplies_articles_equip

In [9]:
df = ddf[['contract_award_unique_key','award_id_piid', 'parent_award_agency_name', 'total_dollars_obligated','period_of_performance_potential_end_date', 'awarding_sub_agency_name', 'awarding_office_name', 'recipient_name', 'award_description','naics_code' ]].compute()

In [12]:
import sqlite3
conn = sqlite3.connect('usaspending_small.db')
df.to_sql('usaspending_small.db', con=conn)

In [10]:
%%time
df_head = ddf.head()

Wall time: 5 s


In [28]:
pd.set_option('display.max_columns', None)
df_head

,contract_transaction_unique_key,contract_award_unique_key,award_id_piid,modification_number,transaction_number,parent_award_agency_id,parent_award_agency_name,parent_award_id_piid,parent_award_modification_number,federal_action_obligation,total_dollars_obligated,base_and_exercised_options_value,current_total_value_of_award,base_and_all_options_value,potential_total_value_of_award,disaster_emergency_fund_codes_for_overall_award,outlayed_amount_funded_by_COVID-19_supplementals_for_overall_aw,obligated_amount_funded_by_COVID-19_supplementals_for_overall_a,action_date,action_date_fiscal_year,period_of_performance_start_date,period_of_performance_current_end_date,period_of_performance_potential_end_date,ordering_period_end_date,solicitation_date,awarding_agency_code,awarding_agency_name,awarding_sub_agency_code,awarding_sub_agency_name,awarding_office_code,awarding_office_name,funding_agency_code,funding_agency_name,funding_sub_agency_code,funding_sub_agency_name,funding_office_code,funding_office_name,treasury_accounts_funding_this_award,federal_accounts_funding_this_award,object_classes_funding_this_award,program_activities_funding_this_award,foreign_funding,foreign_funding_description,sam_exception,sam_exception_description,recipient_duns,recipient_name,recipient_doing_business_as_name,cage_code,recipient_parent_duns,recipient_parent_name,recipient_country_code,recipient_country_name,recipient_address_line_1,recipient_address_line_2,recipient_city_name,recipient_county_name,recipient_state_code,recipient_state_name,recipient_zip_4_code,recipient_congressional_district,recipient_phone_number,recipient_fax_number,primary_place_of_performance_country_code,primary_place_of_performance_country_name,primary_place_of_performance_city_name,primary_place_of_performance_county_name,primary_place_of_performance_state_code,primary_place_of_performance_state_name,primary_place_of_performance_zip_4,primary_place_of_performance_congressional_district,award_or_idv_flag,award_type_code,award_type,idv_type_code,idv_type,multiple_or_single_award_idv_code,multiple_or_single_award_idv,type_of_idc_code,type_of_idc,type_of_contract_pricing_code,type_of_contract_pricing,award_description,action_type_code,action_type,solicitation_identifier,number_of_actions,inherently_governmental_functions,inherently_governmental_functions_description,product_or_service_code,product_or_service_code_description,contract_bundling_code,contract_bundling,dod_claimant_program_code,dod_claimant_program_description,naics_code,naics_description,recovered_materials_sustainability_code,recovered_materials_sustainability,domestic_or_foreign_entity_code,domestic_or_foreign_entity,dod_acquisition_program_code,dod_acquisition_program_description,information_technology_commercial_item_category_code,information_technology_commercial_item_category,epa_designated_product_code,epa_designated_product,country_of_product_or_service_origin_code,country_of_product_or_service_origin,place_of_manufacture_code,place_of_manufacture,subcontracting_plan_code,subcontracting_plan,extent_competed_code,extent_competed,solicitation_procedures_code,solicitation_procedures,type_of_set_aside_code,type_of_set_aside,evaluated_preference_code,evaluated_preference,research_code,research,fair_opportunity_limited_sources_code,fair_opportunity_limited_sources,other_than_full_and_open_competition_code,other_than_full_and_open_competition,number_of_offers_received,commercial_item_acquisition_procedures_code,commercial_item_acquisition_procedures,small_business_competitiveness_demonstration_program,simplified_procedures_for_certain_commercial_items_code,simplified_procedures_for_certain_commercial_items,a76_fair_act_action_code,a76_fair_act_action,fed_biz_opps_code,fed_biz_opps,local_area_set_aside_code,local_area_set_aside,price_evaluation_adjustment_preference_percent_difference,clinger_cohen_act_planning_code,clinger_cohen_act_planning,materials_supplies_articles_equipment_code,materials_supplies_articles_equip

In [9]:
%%time
ddf.tail()

Wall time: 1.91 s


,contract_transaction_unique_key,contract_award_unique_key,award_id_piid,modification_number,transaction_number,parent_award_agency_id,parent_award_agency_name,parent_award_id_piid,parent_award_modification_number,federal_action_obligation,...,highly_compensated_officer_2_name,highly_compensated_officer_2_amount,highly_compensated_officer_3_name,highly_compensated_officer_3_amount,highly_compensated_officer_4_name,highly_compensated_officer_4_amount,highly_compensated_officer_5_name,highly_compensated_officer_5_amount,usaspending_permalink,last_modified_date
12924,4732_4730_47QSEA21F6U44_0_GS02FW0003_0,CONT_AWD_47QSEA21F6U44_4732_GS02FW0003_4730,47QSEA21F6U44,0,0,4730,FEDERAL ACQUISITION SERVICE,GS02FW0003,PW0077,41.01,...,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,https://www.usaspending.gov/award/CONT_AWD_47Q...,2021-05-26 06:44:53
12925,4732_4732_47QSCC21F1YVW_0_47QSCC20A0005_0,CONT_AWD_47QSCC21F1YVW_4732_47QSCC20A0005_4732,47QSCC21F1YVW,0,0,4732,FEDERAL ACQUISITION SERVICE,47QSCC20A0005,PS0001,23.90,...,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,https://www.usaspending.gov/award/CONT_AWD_47Q...,2021-03-08 15:52:46
12926,9700_9700_SPE2DX21F7D0G_0_SPE2DX15D8003_0,CONT_AWD_SPE2DX21F7D0G_9700_SPE2DX15D8003_9700,SPE2DX21F7D0G,0,0,9700,DEPT OF DEFENSE,SPE2DX15D8003,0,13257.12,...,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,https://www.usaspending.gov/award/CONT_AWD_SPE...,2021-05-17 21:34:28
12927,6920_-NONE-_697DCK21A00015_0_-NONE-_-NONE-,CONT_IDV_697DCK21A00015_6920,697DCK21A00015,0,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,0.00,...,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,https://www.usaspending.gov/award/CONT_IDV_697...,2021-07-20 08:54:20
12928,9700_9700_SPE4A620FE9HQ_P00001_SPE4AX16D9010_0,CONT_AWD_SPE4A620FE9HQ_9700_SPE4AX16D9010_9700,SPE4A620FE9HQ,P00001,0,9700,DEPT OF DEFENSE,SPE4AX16D9010,0,0.00,...,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,https://www.usaspending.gov/award/CONT_AWD_SPE...,2021-03-05 11:25:08


In [11]:
# %%time
# ddf.contract_transaction_unique_key.count().compute()

Wall time: 6min 55s


18383722

In [10]:
list[ddf.columns.values]
    

list[array(['contract_transaction_unique_key', 'contract_award_unique_key',
       'award_id_piid', 'modification_number', 'transaction_number',
       'parent_award_agency_id', 'parent_award_agency_name',
       'parent_award_id_piid', 'parent_award_modification_number',
       'federal_action_obligation', 'total_dollars_obligated',
       'base_and_exercised_options_value', 'current_total_value_of_award',
       'base_and_all_options_value', 'potential_total_value_of_award',
       'disaster_emergency_fund_codes_for_overall_award',
       'outlayed_amount_funded_by_COVID-19_supplementals_for_overall_aw',
       'obligated_amount_funded_by_COVID-19_supplementals_for_overall_a',
       'action_date', 'action_date_fiscal_year',
       'period_of_performance_start_date',
       'period_of_performance_current_end_date',
       'period_of_performance_potential_end_date',
       'ordering_period_end_date', 'solicitation_date',
       'awarding_agency_code', 'awarding_agency_name',
       'a

In [11]:
# organize the 284 columns into some common topics for easy of access
date_fields = [col for col in sorted(ddf.columns) if 'date' in col]
recipient_fields = [col for col in sorted(ddf.columns) if 'recipient' in col]
agency_fields = [col for col in sorted(ddf.columns) if 'agency' in col]
code_fields = [col for col in sorted(ddf.columns) if 'code' in col]
type_fields = [col for col in sorted(ddf.columns) if 'type' in col]
obligation_fields = [col for col in sorted(ddf.columns) if 'obligation' in col]
name_fields = [col for col in sorted(ddf.columns) if 'name' in col]

print(f"Example of obligation_fields: {obligation_fields}")
print(f"Example of date_fields: {date_fields}")

Example of obligation_fields: ['federal_action_obligation']
Example of date_fields: ['action_date', 'action_date_fiscal_year', 'consolidated_contract', 'consolidated_contract_code', 'last_modified_date', 'ordering_period_end_date', 'period_of_performance_current_end_date', 'period_of_performance_potential_end_date', 'period_of_performance_start_date', 'solicitation_date']


In [12]:
def Display_System_Info(): # https://psutil.readthedocs.io/en/latest/#psutil.virtual_memory
    physical_cores = psutil.cpu_count(logical=False) #, psutil.cpu_count(logical=True)
    RAM_total_installed = psutil.virtual_memory()[0] #, psutil.swap_memory()
    RAM_available = psutil.virtual_memory().available #['available']
    dask_workers = int(physical_cores/2)
    print(f"Physical CPU Cores: {physical_cores}, RAM available: {round(RAM_available/1e9)} GB, total RAM: {round(RAM_total_installed/1e9)} GB")
    result_record = {'CPU_Cores_Physical' : physical_cores,
                    'RAM_available' : RAM_available,
                     'RAM_total_installed' : RAM_total_installed,
                    }
    return result_record

Display_System_Info()

Physical CPU Cores: 4, RAM available: 6 GB, total RAM: 17 GB


{'CPU_Cores_Physical': 4,
 'RAM_available': 6399946752,
 'RAM_total_installed': 16744906752}

In [13]:
system_info_now = Display_System_Info() # helper function from above

if system_info_now['RAM_available']/1e9 > 16: # you typically need ~15GB to store a GFY in pandas
    USE_PANDAS = True
    print("You should have enough RAM available to load a single GFY archive into RAM. Try moving data from Dask to pandas.")
else:
    USE_PANDAS = False
    print("You may not have enough RAM available to load a GFY into memory. Use Dask instead of pandas.")
print(f"USE_PANDAS = {USE_PANDAS}")

Physical CPU Cores: 4, RAM available: 6 GB, total RAM: 17 GB
You may not have enough RAM available to load a GFY into memory. Use Dask instead of pandas.
USE_PANDAS = False


## Total Dallar spent as award

## Identify the Top 50 Federal Agents for 2020 and their Federal action oligation

In [25]:
%%time
df_agent = ddf.groupby('parent_award_agency_name')[['federal_action_obligation', 'action_date_fiscal_year']].sum().nlargest(20, 'federal_action_obligation').reset_index().compute()

Wall time: 1min 41s


In [26]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
df_agent.federal_action_obligation /= 1000000000
df_agent.rename(columns={"federal_action_obligation": "federal_action_obligation(billions)"}, inplace = True)
df_agent

,parent_award_agency_name,federal_action_obligation(billions)
0,UNSPECIFIED,216.64
1,DEPT OF DEFENSE,95.07
2,FEDERAL ACQUISITION SERVICE,27.84
3,"VETERANS AFFAIRS, DEPARTMENT OF",15.66
4,NATIONAL AERONAUTICS AND SPACE ADMINISTRATION,8.79
5,NATIONAL INSTITUTES OF HEALTH,4.37
6,UNITED STATES MINT,3.16
7,OFFICE OF PROCUREMENT OPERATIONS,2.90
8,"STATE, DEPARTMENT OF",2.62
9,AGRICULTURAL MARKETING SERVICE,2.26


In [30]:
ddf.parent_award_id_piid.nunique().compute()

84071

In [31]:
ddf.award_id_piid.nunique().compute()

3259642

In [47]:
%%time
df_top_agency = ddf.groupby(['parent_award_agency_name'],sort=True).agg({'parent_award_id_piid':['count']}, ).compute()

Wall time: 1min 56s


In [48]:
df_top_agency

,parent_award_id_piid
,count
parent_award_agency_name,
ADMINISTRATION FOR CHILDREN AND FAMILIES,4
AGENCY FOR HEALTHCARE RESEARCH AND QUALITY,84
AGENCY FOR INTERNATIONAL DEVELOPMENT,1012
AGRICULTURAL MARKETING SERVICE,508
AGRICULTURAL RESEARCH SERVICE,514
...,...
US CENSUS BUREAU,428
US GEOLOGICAL SURVEY,1415


In [50]:
%%time
df_agent_count = ddf.groupby('parent_award_agency_name')[['parent_award_id_piid']].count().nlargest(20, 'parent_award_id_piid').reset_index().compute()

Wall time: 2min 3s


In [51]:
df_agent_count

,parent_award_agency_name,parent_award_id_piid
0,DEPT OF DEFENSE,2017679
1,FEDERAL ACQUISITION SERVICE,1005325
2,UNSPECIFIED,795376
3,"VETERANS AFFAIRS, DEPARTMENT OF",46291
4,NATIONAL AERONAUTICS AND SPACE ADMINISTRATION,26566
5,PUBLIC BUILDINGS SERVICE,16409
6,NATIONAL INSTITUTES OF HEALTH,14757
7,"STATE, DEPARTMENT OF",13776
8,FEDERAL PRISON SYSTEM / BUREAU OF PRISONS,12000
9,FOREST SERVICE,9099


In [52]:
%%time
df_contructor_count = ddf.groupby('recipient_parent_name')[['parent_award_id_piid']].count().nlargest(20, 'parent_award_id_piid').reset_index().compute()

Wall time: 1min 46s


In [53]:
df_contructor_count

,recipient_parent_name,parent_award_id_piid
0,AMERISOURCEBERGEN CORPORATION,190011
1,SCIENCE APPLICATIONS INTERNATIONAL CORPORATION,185934
2,CARDINAL HEALTH INC.,159740
3,NATIONAL INDUSTRIES FOR THE BLIND,130317
4,NOBLE SALES CO. INC.,111527
5,OWENS & MINOR INC.,78248
6,W. W. GRAINGER INC.,74770
7,WRIGHT TOOL COMPANY LLC,66848
8,OFFICE DEPOT INC.,46265
9,MONO MACHINES LLC,42870


## All the contracts awarded by FEMA last 3 years

In [54]:
ddf[ddf.parent_award_agency_name == 'FEDERAL EMERGENCY MANAGEMENT AGENCY'].compute()

,contract_transaction_unique_key,contract_award_unique_key,award_id_piid,modification_number,transaction_number,parent_award_agency_id,parent_award_agency_name,parent_award_id_piid,parent_award_modification_number,federal_action_obligation,total_dollars_obligated,base_and_exercised_options_value,current_total_value_of_award,base_and_all_options_value,potential_total_value_of_award,disaster_emergency_fund_codes_for_overall_award,outlayed_amount_funded_by_COVID-19_supplementals_for_overall_aw,obligated_amount_funded_by_COVID-19_supplementals_for_overall_a,action_date,action_date_fiscal_year,period_of_performance_start_date,period_of_performance_current_end_date,period_of_performance_potential_end_date,ordering_period_end_date,solicitation_date,awarding_agency_code,awarding_agency_name,awarding_sub_agency_code,awarding_sub_agency_name,awarding_office_code,awarding_office_name,funding_agency_code,funding_agency_name,funding_sub_agency_code,funding_sub_agency_name,funding_office_code,funding_office_name,treasury_accounts_funding_this_award,federal_accounts_funding_this_award,object_classes_funding_this_award,program_activities_funding_this_award,foreign_funding,foreign_funding_description,sam_exception,sam_exception_description,recipient_duns,recipient_name,recipient_doing_business_as_name,cage_code,recipient_parent_duns,recipient_parent_name,recipient_country_code,recipient_country_name,recipient_address_line_1,recipient_address_line_2,recipient_city_name,recipient_county_name,recipient_state_code,recipient_state_name,recipient_zip_4_code,recipient_congressional_district,recipient_phone_number,recipient_fax_number,primary_place_of_performance_country_code,primary_place_of_performance_country_name,primary_place_of_performance_city_name,primary_place_of_performance_county_name,primary_place_of_performance_state_code,primary_place_of_performance_state_name,primary_place_of_performance_zip_4,primary_place_of_performance_congressional_district,award_or_idv_flag,award_type_code,award_type,idv_type_code,idv_type,multiple_or_single_award_idv_code,multiple_or_single_award_idv,type_of_idc_code,type_of_idc,type_of_contract_pricing_code,type_of_contract_pricing,award_description,action_type_code,action_type,solicitation_identifier,number_of_actions,inherently_governmental_functions,inherently_governmental_functions_description,product_or_service_code,product_or_service_code_description,contract_bundling_code,contract_bundling,dod_claimant_program_code,dod_claimant_program_description,naics_code,naics_description,recovered_materials_sustainability_code,recovered_materials_sustainability,domestic_or_foreign_entity_code,domestic_or_foreign_entity,dod_acquisition_program_code,dod_acquisition_program_description,information_technology_commercial_item_category_code,information_technology_commercial_item_category,epa_designated_product_code,epa_designated_product,country_of_product_or_service_origin_code,country_of_product_or_service_origin,place_of_manufacture_code,place_of_manufacture,subcontracting_plan_code,subcontracting_plan,extent_competed_code,extent_competed,solicitation_procedures_code,solicitation_procedures,type_of_set_aside_code,type_of_set_aside,evaluated_preference_code,evaluated_preference,research_code,research,fair_opportunity_limited_sources_code,fair_opportunity_limited_sources,other_than_full_and_open_competition_code,other_than_full_and_open_competition,number_of_offers_received,commercial_item_acquisition_procedures_code,commercial_item_acquisition_procedures,small_business_competitiveness_demonstration_program,simplified_procedures_for_certain_commercial_items_code,simplified_procedures_for_certain_commercial_items,a76_fair_act_action_code,a76_fair_act_action,fed_biz_opps_code,fed_biz_opps,local_area_set_aside_code,local_area_set_aside,price_evaluation_adjustment_preference_percent_difference,clinger_cohen_act_planning_code,clinger_cohen_act_planning,materials_supplies_articles_equipment_code,materials_supplies_articles_equip

### How might this process work if the company is interested in where AI ('Artificial Intelligence') Work is the Focus on the award_description

In [55]:
%%time
### How might this process work if the company is interested in where AI ('Artificial Intelligence') Work is the Focus on the award_description

search_terms = ['artificial-intelligence', 
                'artificial intelligence', 
                'deep learning',
                'deep-learning',
                'machine learning',
                'machine-learning',
                'data science', 
                'analytics',
                'predictive analytics',
                'predictive-analytics',
                'big data',
                'big-data',
                'business intelligence',
               ] #AI related
LAMBDA_search_term = lambda x: any(a_search_term.lower() in x.lower() for a_search_term in search_terms)
meta = ('award_description', 'str') # needed for Dask but not for pandas
ddf_Contractor_Explore_AI = ddf[ddf['award_description'].apply(LAMBDA_search_term, meta=meta)].compute() # Dask task so must add .compute()

Wall time: 1min 49s


In [56]:
ddf_Contractor_Explore_AI.head()

,contract_transaction_unique_key,contract_award_unique_key,award_id_piid,modification_number,transaction_number,parent_award_agency_id,parent_award_agency_name,parent_award_id_piid,parent_award_modification_number,federal_action_obligation,total_dollars_obligated,base_and_exercised_options_value,current_total_value_of_award,base_and_all_options_value,potential_total_value_of_award,disaster_emergency_fund_codes_for_overall_award,outlayed_amount_funded_by_COVID-19_supplementals_for_overall_aw,obligated_amount_funded_by_COVID-19_supplementals_for_overall_a,action_date,action_date_fiscal_year,period_of_performance_start_date,period_of_performance_current_end_date,period_of_performance_potential_end_date,ordering_period_end_date,solicitation_date,awarding_agency_code,awarding_agency_name,awarding_sub_agency_code,awarding_sub_agency_name,awarding_office_code,awarding_office_name,funding_agency_code,funding_agency_name,funding_sub_agency_code,funding_sub_agency_name,funding_office_code,funding_office_name,treasury_accounts_funding_this_award,federal_accounts_funding_this_award,object_classes_funding_this_award,program_activities_funding_this_award,foreign_funding,foreign_funding_description,sam_exception,sam_exception_description,recipient_duns,recipient_name,recipient_doing_business_as_name,cage_code,recipient_parent_duns,recipient_parent_name,recipient_country_code,recipient_country_name,recipient_address_line_1,recipient_address_line_2,recipient_city_name,recipient_county_name,recipient_state_code,recipient_state_name,recipient_zip_4_code,recipient_congressional_district,recipient_phone_number,recipient_fax_number,primary_place_of_performance_country_code,primary_place_of_performance_country_name,primary_place_of_performance_city_name,primary_place_of_performance_county_name,primary_place_of_performance_state_code,primary_place_of_performance_state_name,primary_place_of_performance_zip_4,primary_place_of_performance_congressional_district,award_or_idv_flag,award_type_code,award_type,idv_type_code,idv_type,multiple_or_single_award_idv_code,multiple_or_single_award_idv,type_of_idc_code,type_of_idc,type_of_contract_pricing_code,type_of_contract_pricing,award_description,action_type_code,action_type,solicitation_identifier,number_of_actions,inherently_governmental_functions,inherently_governmental_functions_description,product_or_service_code,product_or_service_code_description,contract_bundling_code,contract_bundling,dod_claimant_program_code,dod_claimant_program_description,naics_code,naics_description,recovered_materials_sustainability_code,recovered_materials_sustainability,domestic_or_foreign_entity_code,domestic_or_foreign_entity,dod_acquisition_program_code,dod_acquisition_program_description,information_technology_commercial_item_category_code,information_technology_commercial_item_category,epa_designated_product_code,epa_designated_product,country_of_product_or_service_origin_code,country_of_product_or_service_origin,place_of_manufacture_code,place_of_manufacture,subcontracting_plan_code,subcontracting_plan,extent_competed_code,extent_competed,solicitation_procedures_code,solicitation_procedures,type_of_set_aside_code,type_of_set_aside,evaluated_preference_code,evaluated_preference,research_code,research,fair_opportunity_limited_sources_code,fair_opportunity_limited_sources,other_than_full_and_open_competition_code,other_than_full_and_open_competition,number_of_offers_received,commercial_item_acquisition_procedures_code,commercial_item_acquisition_procedures,small_business_competitiveness_demonstration_program,simplified_procedures_for_certain_commercial_items_code,simplified_procedures_for_certain_commercial_items,a76_fair_act_action_code,a76_fair_act_action,fed_biz_opps_code,fed_biz_opps,local_area_set_aside_code,local_area_set_aside,price_evaluation_adjustment_preference_percent_difference,clinger_cohen_act_planning_code,clinger_cohen_act_planning,materials_supplies_articles_equipment_code,materials_supplies_articles_equip

In [59]:
ddf_Contractor_Explore_AI.sort_values(by = 'period_of_performance_potential_end_date')

,contract_transaction_unique_key,contract_award_unique_key,award_id_piid,modification_number,transaction_number,parent_award_agency_id,parent_award_agency_name,parent_award_id_piid,parent_award_modification_number,federal_action_obligation,total_dollars_obligated,base_and_exercised_options_value,current_total_value_of_award,base_and_all_options_value,potential_total_value_of_award,disaster_emergency_fund_codes_for_overall_award,outlayed_amount_funded_by_COVID-19_supplementals_for_overall_aw,obligated_amount_funded_by_COVID-19_supplementals_for_overall_a,action_date,action_date_fiscal_year,period_of_performance_start_date,period_of_performance_current_end_date,period_of_performance_potential_end_date,ordering_period_end_date,solicitation_date,awarding_agency_code,awarding_agency_name,awarding_sub_agency_code,awarding_sub_agency_name,awarding_office_code,awarding_office_name,funding_agency_code,funding_agency_name,funding_sub_agency_code,funding_sub_agency_name,funding_office_code,funding_office_name,treasury_accounts_funding_this_award,federal_accounts_funding_this_award,object_classes_funding_this_award,program_activities_funding_this_award,foreign_funding,foreign_funding_description,sam_exception,sam_exception_description,recipient_duns,recipient_name,recipient_doing_business_as_name,cage_code,recipient_parent_duns,recipient_parent_name,recipient_country_code,recipient_country_name,recipient_address_line_1,recipient_address_line_2,recipient_city_name,recipient_county_name,recipient_state_code,recipient_state_name,recipient_zip_4_code,recipient_congressional_district,recipient_phone_number,recipient_fax_number,primary_place_of_performance_country_code,primary_place_of_performance_country_name,primary_place_of_performance_city_name,primary_place_of_performance_county_name,primary_place_of_performance_state_code,primary_place_of_performance_state_name,primary_place_of_performance_zip_4,primary_place_of_performance_congressional_district,award_or_idv_flag,award_type_code,award_type,idv_type_code,idv_type,multiple_or_single_award_idv_code,multiple_or_single_award_idv,type_of_idc_code,type_of_idc,type_of_contract_pricing_code,type_of_contract_pricing,award_description,action_type_code,action_type,solicitation_identifier,number_of_actions,inherently_governmental_functions,inherently_governmental_functions_description,product_or_service_code,product_or_service_code_description,contract_bundling_code,contract_bundling,dod_claimant_program_code,dod_claimant_program_description,naics_code,naics_description,recovered_materials_sustainability_code,recovered_materials_sustainability,domestic_or_foreign_entity_code,domestic_or_foreign_entity,dod_acquisition_program_code,dod_acquisition_program_description,information_technology_commercial_item_category_code,information_technology_commercial_item_category,epa_designated_product_code,epa_designated_product,country_of_product_or_service_origin_code,country_of_product_or_service_origin,place_of_manufacture_code,place_of_manufacture,subcontracting_plan_code,subcontracting_plan,extent_competed_code,extent_competed,solicitation_procedures_code,solicitation_procedures,type_of_set_aside_code,type_of_set_aside,evaluated_preference_code,evaluated_preference,research_code,research,fair_opportunity_limited_sources_code,fair_opportunity_limited_sources,other_than_full_and_open_competition_code,other_than_full_and_open_competition,number_of_offers_received,commercial_item_acquisition_procedures_code,commercial_item_acquisition_procedures,small_business_competitiveness_demonstration_program,simplified_procedures_for_certain_commercial_items_code,simplified_procedures_for_certain_commercial_items,a76_fair_act_action_code,a76_fair_act_action,fed_biz_opps_code,fed_biz_opps,local_area_set_aside_code,local_area_set_aside,price_evaluation_adjustment_preference_percent_difference,clinger_cohen_act_planning_code,clinger_cohen_act_planning,materials_supplies_articles_equipment_code,materials_supplies_articles_equip

## Top 20 recipient awarding sub agency name for the year 2020

In [57]:
ddf_Contractor_Explore_AI.groupby(['awarding_sub_agency_name'])['federal_action_obligation'].sum().sort_values(ascending=False).reset_index().head(20)

,awarding_sub_agency_name,federal_action_obligation
0,CENTERS FOR MEDICARE AND MEDICAID SERVICES,71131653.35
1,DEPT OF THE AIR FORCE,69683284.36
2,DEFENSE ADVANCED RESEARCH PROJECTS AGENCY (DA...,53397254.00
3,DEPT OF THE NAVY,53129635.43
4,DEPT OF THE ARMY,52817583.57
5,OFFICE OF ASSISTANT SECRETARY FOR PREPAREDNESS...,50996275.00
6,NATIONAL AERONAUTICS AND SPACE ADMINISTRATION,31489565.54
7,FOOD AND DRUG ADMINISTRATION,31157120.00
8,DEPARTMENTAL OFFICES,27949137.75
9,NATIONAL INSTITUTES OF HEALTH,23280991.63


## Top 20 recipient parent company for the year 2020

In [58]:
ddf_Contractor_Explore_AI.groupby(['recipient_parent_name'])['federal_action_obligation'].sum().sort_values(ascending=False).reset_index().head(20)

,recipient_parent_name,federal_action_obligation
0,PALANTIR TECHNOLOGIES INC.,40049821.98
1,BOOZ ALLEN HAMILTON HOLDING CORPORATION,34716106.64
2,DELOITTE LLP,32667910.09
3,ON ASSIGNMENT INC.,32303583.58
4,THE MITRE CORPORATION,30795016.35
5,INTUITIVE MACHINES INC,25863750.00
6,SPECTRAL MD INC.,20634277.00
7,KONIAG INC.,18119940.40
8,ACCENTURE FEDERAL SERVICES LLC,15955957.39
9,BATTELLE MEMORIAL INSTITUTE INC,13709529.00


In [30]:
2+3

5